ファイル構造
データ置き場
在庫MB
所在管理MB

In [ ]:
#メモリのリセット
%reset

In [1]:
ls

kari.ipynb             データ置き場/


In [2]:
#ライブラリのimport
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

在庫推移MotionBoardよりデータ出力

■目的

■在庫推移MotionBoard
http://10.88.11.58:18081/MBSF/main?id=b9154fd0-6fe6-d153-7d17-053da4c333ea&mbid=fidtnjir4nq3ja63cqcaaafmoqcx4&boardpath=%2FMSR%2F%E8%87%AA%E5%8B%95%E3%83%A9%E3%83%83%E3%82%AF%2FT403%E7%89%A9%E6%B5%81%E6%83%85%E5%A0%B1_%E5%9C%A8%E5%BA%AB%E6%8E%A8%E7%A7%BB

In [41]:
# CSVファイルがあるフォルダーのパス
folder_path_zaikoMB = 'データ置き場/在庫MB'
folder_path_LTMB = 'データ置き場/所在管理MB'
file_path_zaikodata = '中間成果物/在庫MBデータ.csv'
file_path_LTdata = '中間成果物/所在管理MBデータ.csv'

In [35]:
# フォルダー内の全てのCSVファイルを見つける
csv_files = [f for f in os.listdir(folder_path_zaikoMB) if f.endswith('.csv')]
#
print(f"{len(csv_files)}つのファイルが見つかりました！")

# 統合結果を保存するための空のDataFrameを作成
merged_df = pd.DataFrame()
# CSVファイルをDataFrameに読み込んでリストに保存
for file in csv_files:
    file_path = os.path.join(folder_path_zaikoMB, file)
    df = pd.read_csv(file_path, encoding='Shift_JIS')
    merged_df = pd.concat([merged_df, df], ignore_index=True)

# 統合したデータを新しいCSVファイルに保存
with open('中間成果物/在庫MBデータ.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_df.to_csv(f)

#作成したDataFrameの内容を確認
merged_df.head(5)

2つのファイルが見つかりました！


,計測日時,品番,拠点所番地,在庫数（箱）,入庫数（箱）,出庫数（箱）
0,2023/12/10 00:00:00,019120LC030,30113,11.0,0.0,0.0
1,2023/12/10 00:00:00,019120LC050,20115,5.0,0.0,0.0
2,2023/12/10 00:00:00,019120LC060,30116,10.0,0.0,0.0
3,2023/12/10 00:00:00,019120LC070,20117,11.0,0.0,0.0
4,2023/12/10 00:00:00,019128GA010,50460,26.0,0.0,0.0


所在管理MOtionBoardよりデータ出力

➀「サブフィルターを表示」押下
➁1Yを設定
➂SHIftjisでダウンロード

■所在管理MotionBoard
http://10.88.11.58:18081/MBSF/main?id=8045ec22-9e62-92c7-5e1e-077d35e5dc33&mbid=fidepnyfuff3ra63kitaaae3ywjmi&boardpath=%2FMSR%2F%E8%87%AA%E5%8B%95%E3%83%A9%E3%83%83%E3%82%AF%2FT403%E7%89%A9%E6%B5%81%E6%83%85%E5%A0%B1_v2

In [80]:
# フォルダー内の全てのCSVファイルを見つける
csv_files = [f for f in os.listdir(folder_path_LTMB) if f.endswith('.csv')]
#
print(f"{len(csv_files)}つのファイルが見つかりました！")

# 統合結果を保存するための空のDataFrameを作成
merged_df = pd.DataFrame()
# CSVファイルをDataFrameに読み込んでリストに保存
for file in csv_files:
    file_path = os.path.join(folder_path_LTMB, file)
    df = pd.read_csv(file_path, encoding='Shift_JIS',dtype=str)
    merged_df = pd.concat([merged_df, df], ignore_index=True)

# 特定の列を削除
merged_df = merged_df.drop("伝票番号", axis=1)
merged_df = merged_df.drop("拠点所番地", axis=1)
merged_df = merged_df.drop("組立投入日時", axis=1)
merged_df = merged_df.drop("発注取消日時", axis=1)
merged_df = merged_df.drop("発注〜組立LT", axis=1)
merged_df = merged_df.drop("長期滞留フラグ", axis=1)

# 特定の文字列を含む行を削除する
target_string = "< NULL >"
merged_df = merged_df[~merged_df.apply(lambda x: x.str.contains(target_string)).any(axis=1)]

# '回収日'をdatatime型に変換
merged_df['回収日時'] = pd.to_datetime(merged_df['回収日時'], errors='coerce')

# ユーザーからの入力を受け付けて特定の日付範囲のDataFrameを取り出す
start_date_input = input("開始日を入力してください（YYYY-MM-DD）: ")
end_date_input = input("終了日を入力してください（YYYY-MM-DD）: ")

# 入力された日付をdatetime型に変換
start_date = pd.to_datetime(start_date_input).date()
end_date = pd.to_datetime(end_date_input).date()

# 特定の日付範囲のDataFrameを取り出す
merged_df = merged_df[(merged_df["回収日時"].dt.date >= start_date) & (merged_df["回収日時"].dt.date <= end_date)]

# 統合したデータを新しいCSVファイルに保存
with open('中間成果物/所在管理MBデータ.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_df.to_csv(f)

#作成したDataFrameの内容を確認
merged_df.head(5)


1つのファイルが見つかりました！


開始日を入力してください（YYYY-MM-DD）:  2023-12-01
終了日を入力してください（YYYY-MM-DD）:  2023-12-20


,かんばんシリアル,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,納入日,納入便,発注日時,...,検収日時,順立装置入庫日時,順立装置出庫日時,回収日時,発注〜印刷LT,発注〜検収LT,発注〜順立装置入庫LT,発注〜順立装置出庫LT,発注〜回収LT,更新日時
6837,1Z23M00048347,9020156A001,ﾌﾟﾚｰﾄﾜｯｼｬ,100,（株）ムロコーポレーション,NaN,回収済,2023/12/19,01,2023/12/19 06:03:22,...,2023/12/20 02:37:06,2023/12/20 06:26:32,2023/12/20 09:57:38,2023-12-20 23:33:17,0.1,0.86,1.02,1.16,1.73,2023/12/20 23:33:17
6838,1Z23M00046187,9020156A001,ﾌﾟﾚｰﾄﾜｯｼｬ,100,（株）ムロコーポレーション,NaN,回収済,2023/12/19,01,2023/12/19 06:03:22,...,2023/12/20 02:37:06,2023/12/20 06:18:18,2023/12/20 15:47:20,2023-12-20 23:33:17,0.1,0.86,1.01,1.41,1.73,2023/12/20 23:33:17
6839,1Z23M00044593,9056463A033,ｼﾑ,100,（株）ムロコーポレーション,NaN,回収済,2023/12/18,01,2023/12/16 06:03:35,...,2023/12/19 02:36:43,2023/12/19 08:49:39,2023/12/19 17:25:23,2023-12-20 23:33:17,2.1,2.86,3.12,3.47,4.73,2023/12/20 23:33:17
6840,1Z23M00043636,35847ECB020,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,70,（株）メタルテック,NaN,回収済,2023/12/20,01,2023/12/16 06:47:00,...,2023/12/20 07:13:21,2023/12/20 09:24:50,2023/12/20 11:51:39,2023-12-20 23:33:17,2.09,4.02,4.11,4.21,4.7,2023/12/20 23:33:17
6841,1Z23M00043122,35847ECE010,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,40,（株）メタルテック,NaN,回収済,2023/12/20,01,2023/12/16 06:47:00,...,2023/12/20 07:13:21,2023/12/20 09:27:11,2023/12/20 16:10:22,2023-12-20 23:33:17,2.09,4.02,4.11,4.39,4.7,2023/12/20 23:33:17


In [84]:
#
LTdata = pd.read_csv(file_path_LTdata,encoding='shift_jis')
zaikodata = pd.read_csv(file_path_zaikodata,encoding='shift_jis')

# '回収日'をdatatime型に変換
LTdata['回収日時'] = pd.to_datetime(merged_df['回収日時'], errors='coerce')

# ユーザーからの入力を受け付けて日付の粒度を決定する
time_scope_input = input("AIモデルに入力するデータの粒度を決定してください（日単位or週単位）: ")

if time_scope_input == "日単位":
    print("日単位でデータを作成します！")
elif time_scope_input == "週単位": 
    print("週単位でデータを作成します！")
    LTdata['年'] = LTdata['回収日時'].dt.year
    LTdata['週番号'] = LTdata['回収日時'].dt.isocalendar().week
    #グループ化し、各グループのLT中央値を計算
    weekly_LTdata_1 = LTdata.groupby(['品番', '仕入先名','年','週番号']).agg({'発注〜印刷LT': 'median','発注〜検収LT': 'median','発注〜順立装置入庫LT': 'median', '発注〜順立装置出庫LT': 'median', '発注〜回収LT': 'median'})
    #グループ化し、各グループの行数（かんばん数）をカウント
    weekly_LTdata_2 = LTdata.groupby(['品番','仕入先名','年', '週番号']).size().reset_index(name='回収かんばん数')
    #データを統合
    merged_weekly_LTdata = pd.merge(weekly_LTdata_1, weekly_LTdata_2, on=['品番', '仕入先名','年','週番号'], how='outer')
    #merged_weekly_LTdata.head(5)
else:
    print("エラー）週単位or日単位で入力してください")
    

AIモデルに入力するデータの粒度を決定してください（日単位or週単位）:  週単位


週単位でデータを作成します！


In [85]:
merged_weekly_LTdata.head(5)

,品番,仕入先名,年,週番号,発注〜印刷LT,発注〜検収LT,発注〜順立装置入庫LT,発注〜順立装置出庫LT,発注〜回収LT,回収かんばん数
0,019128GA010,石黒ゴム工業（株）,2023.0,50,0.320,1.94,2.23,4.260,4.960,79
1,019128GA010,石黒ゴム工業（株）,2023.0,51,0.100,1.95,2.24,5.135,5.880,36
2,01912ECB010,石黒ゴム工業（株）,2023.0,50,0.325,1.94,2.23,5.035,5.415,26
3,01912ECB010,石黒ゴム工業（株）,2023.0,51,0.330,1.94,2.19,4.990,5.750,13
4,01912ECB040,愛産樹脂工業（株）,2023.0,50,0.000,2.96,2.96,5.835,6.630,94


In [2]:
conda update matplotlib

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /anaconda/anaconda3

  added / updated specs:
    - matplotlib


The following packages will be REMOVED:

  importlib_resources-5.2.0-pyhd3eb1b0_1
  matplotlib-base-3.1.3-py37h9aa3819_0
  pkgutil-resolve-name-1.3.10-py37hecd8cb5_0

The following packages will be SUPERSEDED by a higher-priority channel:

  jsonschema         pkgs/main/osx-64::jsonschema-4.17.3-p~ --> pkgs/main/noarch::jsonschema-3.2.0-pyhd3eb1b0_2 None
  seaborn            pkgs/main/osx-64::seaborn-0.12.2-py37~ --> pkgs/main/noarch::seaborn-0.11.2-pyhd3eb1b0_0 None


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [3]:
conda update ipython

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /anaconda/anaconda3

  added / updated specs:
    - ipython


The following NEW packages will be INSTALLED:

  importlib_resourc~ pkgs/main/noarch::importlib_resources-5.2.0-pyhd3eb1b0_1 None
  matplotlib-base    pkgs/main/osx-64::matplotlib-base-3.1.3-py37h9aa3819_0 None
  pkgutil-resolve-n~ pkgs/main/osx-64::pkgutil-resolve-name-1.3.10-py37hecd8cb5_0 None

The following packages will be UPDATED:

  jsonschema         pkgs/main/noarch::jsonschema-3.2.0-py~ --> pkgs/main/osx-64::jsonschema-4.17.3-py37hecd8cb5_0 None
  seaborn            pkgs/main/noarch::seaborn-0.11.2-pyhd~ --> pkgs/main/osx-64::seaborn-0.12.2-py37hecd8cb5_0 None


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Retrieving notices: 

In [75]:
merged_df.isnull().sum()
#merged_df.isnull().info()

かんばんシリアル           0
品番                 0
品名                 0
収容数                0
仕入先名               0
仕入先工場名         36386
ステータス              0
納入日                0
納入便                0
発注日時               0
印刷日時               0
検収日時               0
順立装置入庫日時           0
順立装置出庫日時           0
回収日時               0
発注〜印刷LT            0
発注〜検収LT            0
発注〜順立装置入庫LT        0
発注〜順立装置出庫LT        0
発注〜組立LT            0
発注〜回収LT            0
更新日時               0
dtype: int64

In [57]:
merged_df.dtypes

かんばんシリアル               object
伝票番号                   object
拠点所番地                  object
品番                     object
品名                     object
収容数                    object
仕入先名                   object
仕入先工場名                 object
ステータス                  object
納入日                    object
納入便                    object
発注日時                   object
印刷日時                   object
検収日時                   object
順立装置入庫日時               object
順立装置出庫日時               object
組立投入日時                 object
回収日時           datetime64[ns]
発注取消日時                 object
発注〜印刷LT                object
発注〜検収LT                object
発注〜順立装置入庫LT            object
発注〜順立装置出庫LT            object
発注〜組立LT                object
発注〜回収LT                object
更新日時                   object
長期滞留フラグ                object
dtype: object

In [70]:
merged_df.head(5)

,かんばんシリアル,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,納入日,納入便,発注日時,...,順立装置入庫日時,順立装置出庫日時,回収日時,発注〜印刷LT,発注〜検収LT,発注〜順立装置入庫LT,発注〜順立装置出庫LT,発注〜組立LT,発注〜回収LT,更新日時
